In [21]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
%autoreload 2

In [65]:
from preprocessing import preprocess_sick, preprocess_grid, preprocess_vehicle

In [70]:
df_X, df_y = preprocess_vehicle()
df_X.describe()

,COMPACTNESS,CIRCULARITY,DISTANCE_CIRCULARITY,RADIUS_RATIO,PR.AXIS_ASPECT_RATIO,MAX.LENGTH_ASPECT_RATIO,SCATTER_RATIO,ELONGATEDNESS,PR.AXIS_RECTANGULARITY,MAX.LENGTH_RECTANGULARITY,SCALED_VARIANCE_MAJOR,SCALED_VARIANCE_MINOR,SCALED_RADIUS_OF_GYRATION,SKEWNESS_ABOUT_MAJOR,SKEWNESS_ABOUT_MINOR,KURTOSIS_ABOUT_MAJOR,KURTOSIS_ABOUT_MINOR,HOLLOWS_RATIO
count,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000,846.000000
mean,0.449532,0.456219,0.584565,0.283585,0.161471,0.123913,0.371498,0.426680,0.298562,0.428555,0.308554,0.306848,0.413228,0.177134,0.289867,0.307300,0.431087,0.487746
std,0.179010,0.237303,0.219049,0.146167,0.086684,0.086815,0.217287,0.223187,0.216012,0.207366,0.165236,0.211862,0.204695,0.098513,0.223561,0.217835,0.205465,0.247960
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.304348,0.269231,0.416667,0.161572,0.109890,0.094340,0.223856,0.200000,0.166667,0.271429,0.194737,0.160971,0.251572,0.105263,0.090909,0.121951,0.266667,0.308333
50%,0.434783,0.423077,0.555556,0.275109,0.153846,0.113208,0.294118,0.485714,0.250000,0.400000,0.255263,0.215827,0.402516,0.164474,0.272727,0.268293,0.400000,0.533333
75%,0.586957,0.615385,0.805556,0.397380,0.197802,0.150943,0.562092,0.571429,0.500000,0.585714,0.457895,0.483213,0.559748,0.210526,0.409091,0.463415,0.566667,0.666667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
